## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv('/Users/clarapotts/Desktop/Weather_Database/WeatherPy_Database.csv', index_col=[0])
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Namibe,AO,-15.1961,12.1522,63.01,77,0,4.25,clear sky
1,Jamestown,US,42.0970,-79.2353,74.79,94,100,9.22,overcast clouds
2,Lasa,IT,46.6166,10.7002,68.68,76,41,1.34,scattered clouds
3,Ilulissat,GL,69.2167,-51.1000,50.02,71,100,3.44,light rain
4,Port Elizabeth,ZA,-33.9180,25.5701,52.09,92,0,9.22,clear sky
5,Carnarvon,AU,-24.8667,113.6333,68.49,85,100,19.95,overcast clouds
6,Suez,EG,29.9737,32.5263,79.79,57,0,10.02,clear sky
7,Khatanga,RU,71.9667,102.5000,59.07,43,100,19.80,overcast clouds
8,Ponta Do Sol,PT,32.6667,-17.1000,72.01,68,9,5.61,clear sky
9,Nisia Floresta,BR,-6.0911,-35.2086,75.58,95,93,6.02,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

trips_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
trips_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Jamestown,US,42.0970,-79.2353,74.79,94,100,9.22,overcast clouds
6,Suez,EG,29.9737,32.5263,79.79,57,0,10.02,clear sky
8,Ponta Do Sol,PT,32.6667,-17.1000,72.01,68,9,5.61,clear sky
9,Nisia Floresta,BR,-6.0911,-35.2086,75.58,95,93,6.02,light rain
10,Albany,US,42.6001,-73.9662,79.21,76,21,4.56,few clouds
11,San Juan De Uraba,CO,8.7592,-76.5297,76.14,91,98,7.27,overcast clouds
13,Tautira,PF,-17.7333,-149.1500,75.42,77,7,10.63,light rain
19,Ende,ID,-8.8432,121.6623,81.41,69,82,4.97,broken clouds
21,Katsuura,JP,35.1333,140.3000,80.47,81,79,8.93,broken clouds
22,Rikitea,PF,-23.1203,-134.9692,73.62,74,1,5.03,light rain


In [5]:
# 4a. Determine if there are any empty rows.
trips_df.count()

City                   398
Country                398
Lat                    398
Lng                    398
Max Temp               398
Humidity               398
Cloudiness             398
Wind Speed             398
Current Description    398
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#NA - but if I needed to drop rows I would use the .dropna() on the trip_cities_df 

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = trips_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jamestown,US,74.79,overcast clouds,42.0970,-79.2353,
6,Suez,EG,79.79,clear sky,29.9737,32.5263,
8,Ponta Do Sol,PT,72.01,clear sky,32.6667,-17.1000,
9,Nisia Floresta,BR,75.58,light rain,-6.0911,-35.2086,
10,Albany,US,79.21,few clouds,42.6001,-73.9662,
11,San Juan De Uraba,CO,76.14,overcast clouds,8.7592,-76.5297,
13,Tautira,PF,75.42,light rain,-17.7333,-149.1500,
19,Ende,ID,81.41,broken clouds,-8.8432,121.6623,
21,Katsuura,JP,80.47,broken clouds,35.1333,140.3000,
22,Rikitea,PF,73.62,light rain,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jamestown,US,74.79,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
6,Suez,EG,79.79,clear sky,29.9737,32.5263,Green House
8,Ponta Do Sol,PT,72.01,clear sky,32.6667,-17.1000,Hotel do Campo
9,Nisia Floresta,BR,75.58,light rain,-6.0911,-35.2086,Rancho Crepusculo
10,Albany,US,79.21,few clouds,42.6001,-73.9662,
...,...,...,...,...,...,...,...
804,Darhan,MN,70.30,moderate rain,49.4867,105.9228,Buudai Hotel
805,Charters Towers,AU,70.56,broken clouds,-20.1000,146.2667,Cattleman's Rest Motor Inn
806,Piracuruca,BR,70.20,scattered clouds,-3.9281,-41.7092,Pousada Piracuruca
807,Sambava,MG,70.77,light rain,-14.2667,50.1667,Melrose


In [10]:
# 8a. Create the output File (CSV)
# 8b. Export the City_Data into a csv

hotel_df.to_csv('WeatherPy_Vacation.csv', index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} °F</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]
#locations

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 32.0), zoom_level=1.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))